# 1. Import Library

In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os
from dotenv import load_dotenv
from pathlib import Path

import re

from sentence_transformers import SentenceTransformer

import faiss
from openai import OpenAI

from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

from langsmith import Client

In [2]:
pd.set_option('display.max_colwidth', 500)

# 2. Preparing Data

In [3]:
full_df = pd.read_csv('../artikel/raw_health_articles.csv')
df = full_df[['content', 'source']]
df.head()

,content,source
0,"Ginjal adalah organ vital yang berfungsi menyaring darah, membuang limbah, dan menjaga keseimbangan cairan tubuh. Jika tidak dijaga, fungsi ginjal bisa menurun dan menimbulkan masalah serius. Banyak penyakit ginjal apa saja yang berkembang tanpa gejala pada tahap awal, sehingga penting menjaga kesehatan ginjal sejak dini.\nBerikut ini adalah 5 cara merawat ginjal yang sederhana namun efektif untuk mencegah gejala ginjal bermasalah:\n1. Minum Air Putih yang Cukup\nAsupan cairan yang cukup mem...",Biofarma
1,"Menjaga kesehatan mata sangatlah penting karena mata berperan besar dalam hampir semua aktivitas sehari-hari, mulai dari belajar, bekerja, hingga berinteraksi dengan lingkungan sekitar. Jika kesehatan mata terabaikan, berbagai masalah bisa muncul, mulai dari iritasi ringan, infeksi, hingga penyakit serius yang berisiko menyebabkan kebutaan. Untuk itu, menjaga mata dapat dilakukan melalui pemeriksaan rutin ke dokter dan juga dengan membiasakan pola hidup sehat yang mendukung fungsi penglihata...",Biofarma
2,"Penyebab Campak\nCampak adalah penyakit menular yang disebabkan oleh virus Morbili dan dapat menyebar dengan sangat cepat dari satu orang ke orang lain. Mekanisme utama penularannya adalah melalui percikan air liur (droplet) yang keluar dari mulut atau hidung penderita saat batuk, bersin, atau berbicara. Selain itu, seseorang juga dapat tertular jika menyentuh benda yang sudah terkontaminasi virus lalu menyentuh hidung, mulut, atau mata. Berikut adalah sepuluh faktor penyebab yang paling ber...",Biofarma
3,"Apa itu campak?\nCampak atau rubella adalah penyakit menular yang disebabkan oleh virus dari keluarga Paramyxoviridae dengan genus Morbillivirus. Penyakit ini menyerang saluran pernapasan terlebih dahulu, lalu menimbulkan gejala sistemik seperti demam, batuk, pilek, sakit tenggorokan, mata merah, hingga ruam menyeluruh di kulit. Campak termasuk penyakit yang sangat menular, bisa berpindah dari orang ke orang hanya melalui udara, percikan air liur, atau benda yang terkontaminasi virus.\nApa s...",Biofarma
4,"Apa itu Kesehatan Gigi dan Mulut?\nKesehatan gigi dan mulut adalah kondisi ketika gigi, gusi, lidah, dan seluruh rongga mulut terjaga kebersihan serta fungsinya dengan baik. Mulut merupakan gerbang utama bagi makanan dan minuman yang masuk ke dalam tubuh, sekaligus organ yang berperan penting dalam proses pencernaan. Saat makanan masuk, sebagian akan tertinggal dalam mulut. Jika tidak dibersihkan, sisa tersebut bisa menjadi tempat berkembangnya bakteri. Oleh karena itu, kesehatan mulut tidak...",Biofarma


# 3. Convert to Dictionary

In [4]:
df_dict = df.to_dict()

# 4. Text Cleaning

In [5]:
def clean_text(text):
    # Remove the specific UI noise at the end of the text
    text = text.replace("Bagikan:\nDownload", "")
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [6]:
docs = []
for index, content in df_dict['content'].items():
    cleaned_content = clean_text(content)
    # We create a Document object. 
    # We store the 'index' in metadata so we can trace the source later.
    doc = Document(
        page_content=cleaned_content, 
        metadata={"source_id": index}
    )
    docs.append(doc)

print(f"Loaded {len(docs)} documents.")

Loaded 420 documents.


# 5. Chunking

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""] # Priorities for splitting
)

In [8]:
splits = text_splitter.split_documents(docs)
print(f"Created {len(splits)} chunks from {len(docs)} original documents.")

Created 2712 chunks from 420 original documents.


# 6. Embedding

In [9]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

In [10]:
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=embedding_model
)

# 7. Retrieve Document

In [11]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5} # Retrieve the top 5 most relevant chunks
)

In [12]:
# Test retrieval
query = "Bagaimana cara menjaga kesehatan ginjal?"
retrieved_docs = retriever.invoke(query)

In [13]:
print(f"Query: {query}")
for i, doc in enumerate(retrieved_docs):
    print(f"\n[Result {i+1}] (Source ID: {doc.metadata['source_id']})")
    print(doc.page_content[:200] + "...")

Query: Bagaimana cara menjaga kesehatan ginjal?

[Result 1] (Source ID: 223)
. Beberapa orang yang didiagnosis memiliki kadar kreatinin normal, mungkin saja memiliki penyakit ginjal. Begitu juga sebaliknya, individu dengan kadar kreatinin tinggi belum tentu mengalami penyakit ...

[Result 2] (Source ID: 223)
. Jagalah kesehatan ginjalmu dengan menjaga hidrasi yang cukup, mengendalikan tekanan darah dan kadar gula darah, menjalani gaya hidup sehat, dan melakukan pemeriksaan kesehatan rutin. Jika kamu memil...

[Result 3] (Source ID: 223)
. Tes kreatinin biasanya digunakan untuk memantau kesehatan ginjal dan mendeteksi kemungkinan gangguan ginjal. Apabila setelah pemeriksaan ditemukan adanya gangguan pada organ ginjal, dokter bisa saja...

[Result 4] (Source ID: 37)
Ginjal adalah organ penting yang bekerja tanpa henti untuk menyaring limbah, menjaga keseimbangan elektrolit, dan mengatur tekanan darah. Sayangnya, banyak orang baru menyadari pentingnya fungsi ginja...

[Result 5] (Source I

# 8. Complete Retrieval Augmented Generation (RAG)

In [ ]:
# Check API
env_path = Path('..') / '.env'
load_dotenv(dotenv_path=env_path)

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if GEMINI_API_KEY:
    print("API Key ditemukan!")
else:
    print("API Key masih None. Periksa path file .env")

API Key ditemukan!


In [ ]:
client = OpenAI(
    api_key = GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

def generate_augmented_response(query):
  search_results = retriever.invoke(query)
  context = "\n\n".join([doc.page_content for doc in search_results])

  prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
  response = client.chat.completions.create(
      model="gemini-2.5-flash", # Using model gemini 2.5 flash
      messages=[
        {
          "role": "system",
          "content": "You are a helpful assistant."
        },
        {
          "role": "user",
          "content": prompt
        }
      ],
  )

  return response.choices[0].message.content

## 8.1 Factual Queries

In [16]:
query = "Sebutkan zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata! dan sebutkan bagian mana yang mendukung setiap poin."
response = generate_augmented_response(query)
print(response)

Berikut adalah zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata, beserta bagian teks yang mendukung setiap poin:

1.  **Beta-karoten (diubah menjadi Vitamin A)**
    *   **Manfaat untuk Kesehatan Mata:**
        *   Sangat penting untuk menjaga kesehatan mata, membantu mempertahankan penglihatan yang baik, dan mencegah gangguan mata seperti rabun senja dan degenerasi makula.
        *   Kekurangan vitamin A dapat meningkatkan risiko rabun senja.
        *   Berperan dalam menutrisi mata dan menjaga kelembapannya, sehingga mencegah terjadinya mata kering.
        *   Nutrisi ini penting untuk kesehatan mata.
    *   **Bagian yang Mendukung:**
        *   "Vitamin ini sangat penting untuk menjaga kesehatan mata, membantu mempertahankan penglihatan yang baik, dan mencegah gangguan mata seperti rabun senja dan degenerasi makula."
        *   "Meningkatkan Kesehatan Mata Wortel kaya akan beta-karoten, yang diubah tubuh menjadi vitamin A. Kekurangan vitamin A dapat 

## 8.2 Conceptual Understanding?

In [17]:
query = "Mengapa penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal?"
response = generate_augmented_response(query)
print(response)

Penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal karena:

1.  **Meningkatkan Tekanan Darah:** Asupan garam berlebih dapat meningkatkan tekanan darah.
2.  **Merusak Pembuluh Darah Ginjal:** Tekanan darah tinggi merupakan salah satu faktor utama penyebab penyakit ginjal kronis. Terlalu banyak garam bisa menyebabkan tekanan darah tinggi yang pada akhirnya merusak pembuluh darah di ginjal.
3.  **Menghambat Aliran Darah dan Menurunkan Fungsi Ginjal:** Jika dibiarkan, tekanan darah tinggi dapat menghambat aliran darah ke ginjal dan menurunkan fungsinya.

Dengan membatasi konsumsi garam, tekanan darah dapat terkontrol, sehingga membantu menjaga aliran darah ke ginjal tetap lancar dan melindungi fungsi ginjal.


## 8.3 Question Requiring Synthesis

In [18]:
query = "Analisis dampak merokok terhadap tubuh: organ apa saja yang rusak dan penyakit apa yang mungkin timbul mulai dari kepala (mata) hingga organ dalam (ginjal/jantung)?"
response = generate_augmented_response(query)
print(response)

Berdasarkan teks yang diberikan, dampak merokok terhadap tubuh, mulai dari kepala hingga organ dalam, serta penyakit yang mungkin timbul adalah sebagai berikut:

**I. Kepala dan Indera:**
*   **Otak:**
    *   **Penyakit/Dampak:** Kerusakan otak, penyempitan pembuluh darah otak, penurunan suplai oksigen ke otak.
    *   **Penyebab:** Nikotin yang mempengaruhi pembuluh darah.
*   **Indera (umum):**
    *   **Penyakit/Dampak:** Kerusakan indera.

**II. Mulut, Bibir, Tenggorokan, dan Saluran Pernapasan:**
*   **Mulut & Bibir:**
    *   **Penyakit/Dampak:** Kanker mulut, kanker bibir, pengikisan selaput lendir.
    *   **Penyebab:** Tar, panas dari asap rokok.
*   **Tenggorokan (Laring) & Kerongkongan (Esofagus):**
    *   **Penyakit/Dampak:** Kanker tenggorokan, kanker kerongkongan (risiko 5-10 kali lebih tinggi), pengikisan selaput lendir.
    *   **Penyebab:** Tar.
*   **Saluran Napas & Paru-Paru:**
    *   **Penyakit/Dampak:** Perubahan struktur dan fungsi saluran napas dan jaringan pa

## 8.4 Question Out-Of-Domain

For checking hallucination from LLM generation

In [29]:
query = "Sebutkan pasal yang mengatur tentang pengadaan barang dan jasa dalam UUD!"
response = generate_augmented_response(query)
print(response)

Maaf, informasi mengenai pasal yang mengatur tentang pengadaan barang dan jasa dalam UUD tidak terdapat dalam teks yang Anda berikan. Teks tersebut membahas tentang kolaborasi dan layanan KlikDokter.


## 8.5 Question for Comparation Analysis

In [60]:
query = "gejala diabetes"
response = generate_augmented_response(query)
print(response)

Berdasarkan teks, gejala diabetes (khususnya disebutkan untuk diabetes tipe 1 pada anak, namun dapat menjadi peringatan dini secara umum) meliputi:

*   **Banyak makan** (sering merasa lapar)
*   **Banyak minum** (sering merasa haus)
*   **Banyak kencing** (sering buang air kecil, termasuk mengompol lagi bagi anak yang sudah tidak mengompol)
*   **Berat badan turun drastis**
*   **Mulai loyo** (lemas atau kurang energi)


# 9. Evaluation

## 9.1 Retrieval Evaluation

In [26]:
ground_truth = {
    "Sebutkan zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata! dan sebutkan bagian mana yang mendukung setiap poin.": [78, 96],
    "Mengapa penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal?": [0, 15, 53, 190, 223],
    "Analisis dampak merokok terhadap tubuh: organ apa saja yang rusak dan penyakit apa yang mungkin timbul mulai dari kepala (mata) hingga organ dalam (ginjal/jantung)?": [4, 15, 46, 54, 276]
}

In [ ]:
print(f"--- Retrieval Evaluation (Precision@{k}) ---")

for query, relevant_ids in ground_truth.items():
    # 1. Retrieve
    retrieved_docs = retriever.invoke(query)
    
    # 2. Take ID from metadata
    retrieved_ids = [doc.metadata.get('source_id') for doc in retrieved_docs]
    
    print(f"Query: {query}")
    print(f"   Debug - Retrieved IDs: {retrieved_ids}")
    print(f"   Debug - Target IDs: {relevant_ids}")

    # 3. Calculate Precision
    relevant_retrieved = [i for i in retrieved_ids if i in relevant_ids]
    
    precision = len(relevant_retrieved) / len(retrieved_docs) if retrieved_docs else 0
    results[query] = precision
    
    print(f"Final Precision@{k} Score: {precision:.2f}\n")

--- Retrieval Evaluation (Precision@5) ---
Query: Sebutkan zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata! dan sebutkan bagian mana yang mendukung setiap poin.
   Debug - Retrieved IDs: [96, 96, 96, 78, 96]
   Debug - Target IDs: [78, 96]
Final Precision@5 Score: 1.00

Query: Mengapa penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal?
   Debug - Retrieved IDs: [0, 15, 316, 53, 223]
   Debug - Target IDs: [0, 15, 53, 190, 223]
Final Precision@5 Score: 0.80

Query: Analisis dampak merokok terhadap tubuh: organ apa saja yang rusak dan penyakit apa yang mungkin timbul mulai dari kepala (mata) hingga organ dalam (ginjal/jantung)?
   Debug - Retrieved IDs: [54, 54, 276, 54, 276]
   Debug - Target IDs: [4, 15, 46, 54, 276]
Final Precision@5 Score: 1.00



## 9.2 Langsmith Evaluation

In [53]:
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Health_RAG_Evaluation"
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY

In [54]:
client_ls = Client(api_key=LANGCHAIN_API_KEY)

try:
    list(client_ls.list_datasets(limit=1))
    print("✅ Authentication SUCCESS!")
except Exception as e:
    print("❌ Authentication FAILED!")
    print(e)

✅ Authentication SUCCESS!


In [47]:
# New pipeline for evaluation (same as before but using dict as input)

def rag_pipeline(inputs):
    """
    RAG function that accepts a dict input (from LangSmith)
    and returns a dict output (Answer + Documents).
    """
    query = inputs["question"]
    
    # 1. Retrieve
    retrieved_docs = retriever.invoke(query)
    
    # 2. Contextualize
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    
    # 3. Generate
    prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    
    response = client.chat.completions.create(
      model="gemini-2.5-flash", 
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
      ],
    )
    
    answer_text = response.choices[0].message.content
    
    return {
        "answer": answer_text,
        "documents": retrieved_docs
    }

In [37]:
# Define the Judge LLM (Using Gemini 2.5 Flash as the grader)
llm_judge = ChatOpenAI(
    model="gemini-2.5-flash",
    openai_api_base="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=GEMINI_API_KEY,
    temperature=0
)

### 9.2.1 Correctness: Response vs Reference Answer

In [48]:
# --- 1. Correctness Evaluator ---
class CorrectnessGrade(BaseModel):
    explanation: str = Field(description="Explain your reasoning for the score")
    correct: bool = Field(description="True if the answer is correct, False otherwise.")

def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    prompt = f"""You are a teacher grading a quiz.
    QUESTION: {inputs['question']}
    GROUND TRUTH: {reference_outputs['answer']}
    STUDENT ANSWER: {outputs['answer']}
    
    Grade based on factual accuracy relative to the ground truth.
    """
    grader = llm_judge.with_structured_output(CorrectnessGrade)
    grade = grader.invoke(prompt)
    return grade.correct

### 9.2.2 Groundedness: Response vs Retrieved Docs

In [39]:
# --- 2. Groundedness Evaluator (Hallucination Check) ---
class GroundedGrade(BaseModel):
    explanation: str = Field(description="Explain your reasoning for the score")
    grounded: bool = Field(description="True if the answer is supported by facts, False otherwise.")

def groundedness(inputs: dict, outputs: dict) -> bool:
    # Combine retrieved docs into a single string
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    
    prompt = f"""You are a teacher grading a quiz.
    FACTS: {doc_string}
    STUDENT ANSWER: {outputs['answer']}
    
    Ensure the STUDENT ANSWER is grounded in the FACTS.
    """
    grader = llm_judge.with_structured_output(GroundedGrade)
    grade = grader.invoke(prompt)
    return grade.grounded

### 9.2.4 Run Evaluation

In [58]:
client_ls = Client(api_key=LANGCHAIN_API_KEY)

dataset_name = "Health_RAG_Dataset_V1.1"

# 1. Check if dataset exists, if not create it
if not client_ls.has_dataset(dataset_name=dataset_name):
    dataset = client_ls.create_dataset(dataset_name=dataset_name)
    
    examples = [
        {
            "inputs": {"question": "Sebutkan zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata!"},
            "outputs": {"answer": "Wortel mengandung Beta-karoten yang diubah menjadi Vitamin A, serta Lutein. "},
        },
        {
            "inputs": {"question": "Mengapa penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal?"},
            "outputs": {"answer": "Garam berlebih meningkatkan tekanan darah. Tekanan darah tinggi merusak pembuluh darah ginjal, menghambat aliran darah, dan menurunkan fungsi ginjal."},
        },
        {
            "inputs": {"question": "Sebutkan pasal yang mengatur tentang pengadaan barang dan jasa dalam UUD!"},
            "outputs": {"answer": "Maaf, informasi tersebut tidak tersedia dalam konteks dokumen yang diberikan."},
        }
    ]
    
    client_ls.create_examples(
        dataset_id=dataset.id,
        examples=examples
    )
    print("Dataset created!")
else:
    print("Dataset already exists.")

# 2. Run the Evaluation
results = client_ls.evaluate(
    rag_pipeline,
    data=dataset_name,
    evaluators=[correctness, groundedness],
    experiment_prefix="gemini-rag-test"
)

Dataset created!
View the evaluation results for experiment: 'gemini-rag-test-4b84a0f4' at:
https://smith.langchain.com/o/aa841df8-5022-4144-a456-958c1b95a656/datasets/c8639e6a-ad82-4561-bcf2-d7a9ee70afaf/compare?selectedSessions=f90fb2d2-68e1-439c-a16a-02f6a5cd4a51




3it [00:34, 11.54s/it]


In [59]:
results.to_pandas()

,inputs.question,outputs.answer,outputs.documents,error,reference.answer,feedback.correctness,feedback.groundedness,execution_time,example_id,id
0,Sebutkan pasal yang mengatur tentang pengadaan barang dan jasa dalam UUD!,Informasi mengenai pasal yang mengatur tentang pengadaan barang dan jasa dalam UUD tidak tersedia dalam teks yang diberikan. Teks tersebut membahas tentang kolaborasi dan layanan KlikDokter.,"[page_content='. Acara ini bertujuan untuk memenuhi kebutuhan informasi dan menstimulasi investasi di bidang penyedia layanan kota cerdas. Helatan ini juga diharapkan berperan sebagai sektor pendukung dalam sistem manajemen dan teknologi air bersih, pengelolaan persampahan, limbah, dan sistem transportasi yang semuanya terhubung dalam ekosistem kota cerdas. Tahun ini, KlikDokter memperkenalkan diri sebagai salah satu teknologi untuk mewujudkan smart city. Artikel Lainnya: Tips Memilih Tensim...",None,"Maaf, informasi tersebut tidak tersedia dalam konteks dokumen yang diberikan.",True,True,3.191791,54981987-21d5-4f5b-ae9f-775102938a9d,4419e963-b5bb-41ce-95dd-98eb9a2f7f8f
1,Mengapa penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal?,"Berdasarkan teks, penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal karena:\n\n1. **Garam meningkatkan tekanan darah:** Asupan garam berlebih dapat meningkatkan tekanan darah.\n2. **Tekanan darah tinggi merusak ginjal:** Tekanan darah tinggi merupakan salah satu faktor utama penyebab penyakit ginjal kronis. Terlalu banyak garam bisa menyebabkan tekanan darah tinggi yang pada akhirnya merusak pembuluh darah di ginjal.\n3. **Menghambat aliran darah ke ginjal...","[page_content='. 2. Batasi Konsumsi Garam Asupan garam berlebih dapat meningkatkan tekanan darah. Tekanan darah tinggi merupakan salah satu faktor utama penyebab penyakit ginjal kronis. Mengurangi makanan asin seperti mie instan dan makanan olahan membantu menjaga aliran darah ke ginjal tetap lancar. 3. Jaga Kadar Gula Darah Kadar gula darah tinggi tidak hanya menyebabkan diabetes, tetapi juga bisa merusak pembuluh darah kecil di ginjal. Kondisi ini dapat memicu ciri penyakit ginjal seperti ...",None,"Garam berlebih meningkatkan tekanan darah. Tekanan darah tinggi merusak pembuluh darah ginjal, menghambat aliran darah, dan menurunkan fungsi ginjal.",True,True,8.799139,83714f98-4281-453c-9fba-794730748ea8,d6a19b57-bb12-4c2f-b8f2-7b038d84629e
2,Sebutkan zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata!,Zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata adalah **beta-karoten** dan **vitamin A**.,"[page_content='. Lutein memainkan peran penting dalam menjaga kesehatan mata. Likopen : Ini adalah antioksidan merah terang yang ditemukan dalam sejumlah buah dan sayuran merah, termasuk wortel merah dan ungu. Telah disarankan bahwa likopen mungkin memiliki efek perlindungan terhadap kanker dan penyakit kardiovaskular. Proses memasak wortel memfasilitasi pelepasan likopen dan meningkatkan penyerapannya. Poliasetilena : Penelitian terbaru telah mengidentifikasi senyawa bioaktif dalam wortel y...",None,"Wortel mengandung Beta-karoten yang diubah menjadi Vitamin A, serta Lutein.",False,True,4.708059,b8032bd4-b504-4851-a597-b1984d15c57d,c0263160-e65d-451d-8bac-9833e143bb79
